In [1]:
import csv
import itertools
from collections import defaultdict
from typing import DefaultDict, Dict, List, Set, Tuple

In [2]:
data: DefaultDict[str, DefaultDict[float, DefaultDict[int, DefaultDict[int, List[Tuple[float, float]]]]]] = defaultdict(
    lambda: defaultdict(lambda: defaultdict(lambda: defaultdict(list)))
)
possible: Tuple[Set[str], Set[float], Set[int], Set[int]] = (set(), set(), set(), set())
with open("vrpdfd-summary.csv", "r", encoding="utf-8", newline="") as csvfile:
    csvreader = csv.reader(csvfile, delimiter=",", quotechar="\"")
    csvlines = iter(csvreader)
    next(csvlines)  # Skip header

    for row in csvlines:
        problem = row[0]
        mutation_rate = float(row[3])
        reset_after = int(row[4])
        local_search_batch = int(row[6])
        profit = float(row[7])
        computation_time = float(row[11])

        possible[0].add(problem)
        possible[1].add(mutation_rate)
        possible[2].add(reset_after)
        possible[3].add(local_search_batch)
        data[problem][mutation_rate][reset_after][local_search_batch].append((profit, computation_time))

possible

({'50.10.1',
  '50.10.3',
  '50.20.1',
  '50.20.3',
  '50.30.1',
  '50.30.3',
  '50.40.1',
  '50.40.3'},
 {0.1, 0.5, 0.9},
 {10, 15, 20},
 {20, 50, 80})

In [3]:
sums: DefaultDict[str, List[float]] = defaultdict(list)
for problem, mutation_rate, reset_after, local_search_batch in itertools.product(*possible):
    sums[problem].extend(profit for profit, _ in data[problem][mutation_rate][reset_after][local_search_batch])

average: Dict[str, float] = {}
for problem, s in sums.items():
    average[problem] = sum(s) / len(s)

average

{'50.30.1': -42079.37000000002,
 '50.20.3': 1793.340000000003,
 '50.20.1': 1889.2668518518517,
 '50.10.3': 27271.370185185184,
 '50.40.3': -76136.85574074072,
 '50.10.1': 25204.048888888883,
 '50.40.1': -105186.51425925925,
 '50.30.3': -33908.36944444444}

In [4]:
normalized_data: DefaultDict[str, DefaultDict[float, DefaultDict[int, Dict[int, float]]]] = defaultdict(
    lambda: defaultdict(lambda: defaultdict(dict))
)
max_data: DefaultDict[str, DefaultDict[float, DefaultDict[int, Dict[int, float]]]] = defaultdict(
    lambda: defaultdict(lambda: defaultdict(dict))
)

for problem, mutation_rate, reset_after, local_search_batch in itertools.product(*possible):
    profits = [profit for profit, _ in data[problem][mutation_rate][reset_after][local_search_batch]]
    normalized_data[problem][mutation_rate][reset_after][local_search_batch] = (sum(profits) / len(profits) - average[problem]) / abs(average[problem])
    max_data[problem][mutation_rate][reset_after][local_search_batch] = (max(profits) - average[problem]) / abs(average[problem])

In [5]:
normalized_improved: Dict[Tuple[float, int, int], float] = {}
for mutation_rate, reset_after, local_search_batch in itertools.product(*possible[1:]):
    values: List[float] = [normalized_data[problem][mutation_rate][reset_after][local_search_batch] for problem in possible[0]]
    normalized_improved[mutation_rate, reset_after, local_search_batch] = sum(values) / len(values)

normalized_improved

{(0.1, 10, 80): 0.8391322182421443,
 (0.1, 10, 50): 0.6665294311075854,
 (0.1, 10, 20): 0.3911313214293701,
 (0.1, 20, 80): -0.5772411650973518,
 (0.1, 20, 50): -0.4705495989332569,
 (0.1, 20, 20): -0.4897926954323732,
 (0.1, 15, 80): -0.1390131528922687,
 (0.1, 15, 50): 0.16706562543383202,
 (0.1, 15, 20): -0.09617740637551862,
 (0.5, 10, 80): 0.5496446123796763,
 (0.5, 10, 50): 0.43021926991148723,
 (0.5, 10, 20): 0.5089568158074481,
 (0.5, 20, 80): -0.40502430935417905,
 (0.5, 20, 50): -0.45932520366055246,
 (0.5, 20, 20): -0.5542936811373362,
 (0.5, 15, 80): 0.25265387382508864,
 (0.5, 15, 50): -0.1279979299460501,
 (0.5, 15, 20): 0.0005053252520161419,
 (0.9, 10, 80): 0.5199086864209065,
 (0.9, 10, 50): 0.5998342245510422,
 (0.9, 10, 20): 0.31280797910542674,
 (0.9, 20, 80): -0.6105188320723683,
 (0.9, 20, 50): -0.5182056469481429,
 (0.9, 20, 20): -0.4783965327027651,
 (0.9, 15, 80): -0.13882822215287643,
 (0.9, 15, 50): -0.06522867416416811,
 (0.9, 15, 20): -0.10779633259681064}

In [6]:
max_improved: Dict[Tuple[float, int, int], float] = {}
for mutation_rate, reset_after, local_search_batch in itertools.product(*possible[1:]):
    values: List[float] = [max_data[problem][mutation_rate][reset_after][local_search_batch] for problem in possible[0]]
    max_improved[mutation_rate, reset_after, local_search_batch] = sum(values) / len(values)

max_improved

{(0.1, 10, 80): 1.2509064362425704,
 (0.1, 10, 50): 1.3307506493310872,
 (0.1, 10, 20): 1.142754244149574,
 (0.1, 20, 80): 0.8286691114022222,
 (0.1, 20, 50): 0.9396006633519889,
 (0.1, 20, 20): 0.6951673160628971,
 (0.1, 15, 80): 0.93075251603721,
 (0.1, 15, 50): 1.0111048303950203,
 (0.1, 15, 20): 0.913013587912643,
 (0.5, 10, 80): 1.3792593970560874,
 (0.5, 10, 50): 1.0984095220498984,
 (0.5, 10, 20): 1.0920609967924138,
 (0.5, 20, 80): 0.7527995638325702,
 (0.5, 20, 50): 0.7277535382442883,
 (0.5, 20, 20): 0.10792549707472573,
 (0.5, 15, 80): 1.1337419797682955,
 (0.5, 15, 50): 0.8021534534901515,
 (0.5, 15, 20): 0.9251477871891514,
 (0.9, 10, 80): 1.2564894235372754,
 (0.9, 10, 50): 1.3583536129112825,
 (0.9, 10, 20): 1.2576028928301404,
 (0.9, 20, 80): 0.4462374711562369,
 (0.9, 20, 50): 0.81079676842072,
 (0.9, 20, 20): 0.736642851851607,
 (0.9, 15, 80): 1.2126237301197442,
 (0.9, 15, 50): 1.2161093379625263,
 (0.9, 15, 20): 0.7060792730871599}

In [7]:
max(itertools.product(*possible[1:]), key=normalized_improved.__getitem__)

(0.1, 10, 80)

In [8]:
max(itertools.product(*possible[1:]), key=max_improved.__getitem__)

(0.5, 10, 80)